In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
!pip uninstall umap -y
!pip uninstall umap-learn -y
!pip install umap-learn
import umap as umap

root = "/content/drive"
drive.mount(root,  force_remount=False)
url_dict = {
            'RoBERTa-DE-EN' : 'T-Systems-onsite/cross-en-de-roberta-sentence-transformer',
            'USE_Multilingual':'distiluse-base-multilingual-cased-v1'
}

#@markdown  #Global Parameters

model_type = 'RoBERTa-DE-EN' #@param ['RoBERTa-DE-EN', 'USE_Multilingual']

model_name = url_dict[model_type]
data_name = 'data_'+model_type+'.pkl'
embeddings_name = 'embeddings50_'+model_type
name = "LUSIR_Data_Exploration_7-5-21" #@param {type:'string'}
labeled_data_name = 'data_'+model_type


#@markdown path_name specifies the folder where everything is saved.
path_name = "Output LUSIR/" #@param {type:'string'}
path = root+"/My Drive/"+path_name

name_path = path+name+"/"
output_path = name_path+"out/"
model_path = name_path+"model/"
data_path = path+"preprocessed/"
cluster_path = root+"/My Drive/"+"with_label/50word_with_topic_llama2Model_finale.csv"
data = pd.read_csv(cluster_path)
embeddings = np.load(data_path+embeddings_name+'.npy', mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
embeddings_reduced = pickle.load(open(model_path+name+"finale_reducer.pkl", "rb"))
scaler = StandardScaler()
scaler.fit(embeddings_reduced)
embeddings_reduced = scaler.transform(embeddings_reduced)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.4-py3-none-any.whl size=86770 sha256=a1712eb838ec88386069c61f4834109a8d5cee964a44aad2570468fbe71ba1ac
  Stored in directory: /root/.cache/pip/wheels/fb/66/29/199acf5784d0f7b8add6d466175ab45506c96e386ed5dd0633
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55615 sha256=357ec92baaac4c81511a052451551e6bfb3e39d839bfc4ba606a93bc3057feab
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
Successfully built umap-learn pynndescent
Mounted at /content/drive


### **Classification_SVM**

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

y = data['cluster']
X = embeddings_reduced
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

linear_svm = SVC(kernel='linear')
linear_svm.fit(X_train, y_train)

poly_svm = SVC(kernel='poly', degree=3)  # Polynomial kernel of degree 3
poly_svm.fit(X_train, y_train)

rbf_svm = SVC(kernel='rbf')
rbf_svm.fit(X_train, y_train)

sigmoid_svm = SVC(kernel='sigmoid')
sigmoid_svm.fit(X_train, y_train)

# Make predictions using each classifier
y_linear_predictions = linear_svm.predict(X_test)
y_poly_predictions = poly_svm.predict(X_test)
y_rbf_predictions = rbf_svm.predict(X_test)
y_sigmoid_predictions = sigmoid_svm.predict(X_test)

# Evaluate the performance of each classifier
linear_accuracy = accuracy_score(y_test, y_linear_predictions)
poly_accuracy = accuracy_score(y_test, y_poly_predictions)
rbf_accuracy = accuracy_score(y_test, y_rbf_predictions)
sigmoid_accuracy = accuracy_score(y_test, y_sigmoid_predictions)

random_forest = RandomForestClassifier(random_state=42)
random_forest.fit(X_train, y_train)
y_rf_predictions = random_forest.predict(X_test)
rf_accuracy = accuracy_score(y_test, y_rf_predictions)
print(f"Random Forest Accuracy: {rf_accuracy}")

print("Linear Kernel Accuracy:", linear_accuracy)
print("Polynomial Kernel Accuracy:", poly_accuracy)
print("RBF Kernel Accuracy:", rbf_accuracy)
print("Sigmoid Kernel Accuracy:", sigmoid_accuracy)

print("Linear Kernel classification_report:", classification_report(y_test, y_linear_predictions))
print("Polynomial Kernel classification_report:", classification_report(y_test, y_poly_predictions))
print("RBF Kernel classification_report:", classification_report(y_test, y_rbf_predictions))
print("Sigmoid Kernel classification_report:", classification_report(y_test, y_sigmoid_predictions))


Random Forest Accuracy: 0.974323386537127
Linear Kernel Accuracy: 0.9826509368494102
Polynomial Kernel Accuracy: 0.9201943095072866
RBF Kernel Accuracy: 0.9583622484385843
Sigmoid Kernel Accuracy: 0.9174184594031922
Linear Kernel classification_report:               precision    recall  f1-score   support

           0       0.85      1.00      0.92        17
           1       0.50      1.00      0.67         1
           2       0.00      0.00      0.00         0
           4       0.82      1.00      0.90         9
           5       1.00      1.00      1.00        56
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         9
           9       1.00      1.00      1.00         1
          10       1.00      0.89      0.94         9
          11       1.00      0.96      0.98        27
          12       1.00      1.00      1.00         1
          13       1.00      1.00      1.00        24
          14       0.00      0.00      0.00 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

### **Classification_KNN**

In [4]:
from sklearn.neighbors import KNeighborsClassifier
print("Result of KNN:")
y = data['cluster']
X = embeddings_reduced

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

k = 3  # Number of neighbors to consider
knn_model = KNeighborsClassifier(n_neighbors=k)

# Train the model on the training data
knn_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = knn_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy :", accuracy)

Result of KNN:
Accuracy : 0.9805690492713394


In [5]:
#!pip install tensorflow
#!pip install --upgrade tensorflow

import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
embeddings2=embeddings
# Split your data into training and testing sets
y = data['cluster']
X = embeddings_reduced
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape[1])
# Standardize your features (important for neural networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),  # Input layer
    keras.layers.Dense(X_train.shape[1], activation='relu'),    # Hidden layer with 128 units and ReLU activation
    keras.layers.Dense(X_train.shape[1]/2, activation='relu'),     # Hidden layer with 64 units and ReLU activation
    keras.layers.Dense(len(y.unique()), activation='sigmoid')   # Output layer with 10 units (adjust based on your number of clusters)
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=256, validation_split=0.2, verbose=2,
        callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)],
        validation_data=(X_test, y_test))
# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred_labels = [np.argmax(pred) for pred in y_pred]
accuracy = accuracy_score(y_test, y_pred_labels)
print(f"Neural Network Accuracy: {accuracy}")


90
Epoch 1/100
23/23 - 1s - loss: 4.4711 - accuracy: 0.1223 - val_loss: 3.6868 - val_accuracy: 0.2283 - 836ms/epoch - 36ms/step
Epoch 2/100
23/23 - 0s - loss: 2.8584 - accuracy: 0.4067 - val_loss: 2.0565 - val_accuracy: 0.5913 - 89ms/epoch - 4ms/step
Epoch 3/100
23/23 - 0s - loss: 1.5058 - accuracy: 0.7420 - val_loss: 1.1147 - val_accuracy: 0.8050 - 112ms/epoch - 5ms/step
Epoch 4/100
23/23 - 0s - loss: 0.8682 - accuracy: 0.8740 - val_loss: 0.7358 - val_accuracy: 0.8994 - 109ms/epoch - 5ms/step
Epoch 5/100
23/23 - 0s - loss: 0.5997 - accuracy: 0.9133 - val_loss: 0.5598 - val_accuracy: 0.9098 - 79ms/epoch - 3ms/step
Epoch 6/100
23/23 - 0s - loss: 0.4636 - accuracy: 0.9244 - val_loss: 0.4571 - val_accuracy: 0.9223 - 79ms/epoch - 3ms/step
Epoch 7/100
23/23 - 0s - loss: 0.3769 - accuracy: 0.9374 - val_loss: 0.3934 - val_accuracy: 0.9417 - 92ms/epoch - 4ms/step
Epoch 8/100
23/23 - 0s - loss: 0.3181 - accuracy: 0.9480 - val_loss: 0.3464 - val_accuracy: 0.9417 - 96ms/epoch - 4ms/step
Epoch 9/1